In [14]:
import fastai
from fastai.vision.all import *
from fastai.callback.tracker import EarlyStoppingCallback, SaveModelCallback

import albumentations as A


In [15]:
# Define data path
path = Path('data')

# Set the model directory outside the data folder
model_dir = Path('./models')

# Ensure the directory exists
model_dir.mkdir(parents=True, exist_ok=True)

## Data Augmentations using Albumentations

In [16]:
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2

    def __init__(self, train_aug, valid_aug): store_attr()

    def before_call(self, b, split_idx):
        self.idx = split_idx

    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

def get_train_aug(sz):
    return A.Compose([
        A.RandomResizedCrop(sz, sz),
        A.Transpose(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        A.CoarseDropout(p=0.5),
    ])

def get_valid_aug(sz):
    return A.Compose([
        A.Resize(sz, sz)
    ], p=1.)


## Data Loader
DataLoaders: A fastai class that stores multiple DataLoader objects you pass to it, normally a train and a valid, although it's possible to have as many as you like. The first two are made available as properties.

In [17]:
class DataLoaders(GetAttr):
    def __init__(self, *loaders): self.loaders = loaders
    def __getitem__(self, i): return self.loaders[i]
    train,valid = add_props(lambda i,self: self[i])

With `DataBlock` API you can fully customize every stage of the creation of your `DataLoaders`.

In [18]:
def get_dls(path, sz, bs):
    item_tfms = AlbumentationsTransform(get_train_aug(sz), get_valid_aug(sz))
    batch_tfms = [Normalize.from_stats(*imagenet_stats)]

    dls = ImageDataLoaders.from_folder(
        path,         # Path to your folder containing class subfolders
        valid_pct=0.2, # Split 20% for validation
        seed=42,       # Random seed
        item_tfms=item_tfms, # Apply Albumentations transforms
        batch_tfms=batch_tfms, # Normalization
        bs=bs          # Batch size
    )
    return dls

# Get DataLoaders
dls = get_dls(path, sz=224, bs=64)

<ipython-input-16-17eb338b5a5b>:19: UserWarning: 2 validation errors for InitSchema
scale
  Input should be a valid tuple [type=tuple_type, input_value=224, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/tuple_type
size
  Input should be a valid tuple [type=tuple_type, input_value=224, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/tuple_type
  A.RandomResizedCrop(sz, sz),


ValidationError: 6 validation errors for InitSchema
p
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
scale
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
ratio
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
size
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
interpolation
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
mask_interpolation
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

# Modelling

We don't have a lot of data for our problem, so to train our model, we'll use `RandomResizedCrop` with an image size of 224 px, which is fairly standard for image classification, and default aug_transforms:

In [ ]:
#learn = vision_learner(dls, resnet18, metrics=error_rate)
learn = vision_learner(dls, resnet18, metrics=error_rate, wd=1e-2, cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=2), SaveModelCallback(monitor='valid_loss')])

learn.model_dir = model_dir.absolute()
learn.fine_tune(10)

In [ ]:
learn.save('stage-1')
learn.unfreeze()
lrs = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
learn.fit_one_cycle(10, lrs.valley)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10, nrows=2)

In [19]:
from sklearn.metrics import classification_report

# Get the predictions and actual values for the validation set
def print_classification_report(learn):
    # Get predictions and true labels
    preds, targs = learn.get_preds(dl=learn.dls.valid)

    # Convert predictions to class indices
    pred_classes = preds.argmax(dim=1)

    # Get the list of class labels from the dataloaders
    class_names = learn.dls.vocab

    # Print classification report
    print(classification_report(targs, pred_classes, target_names=class_names))

# Call the function after loading your model or after training
print_classification_report(learn)


NameError: name 'learn' is not defined

In [ ]:
learn.path = model_dir
learn.export('hair-resnet18-model.pkl')